In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
import warnings
warnings.filterwarnings("ignore") # Suppress warnings for
cleaner output

# Step 1: Load the Excel file
file_path = 'interpolated_kerala_data_1991_2011.xlsx' #
Update this path if needed
try:
df = pd.read_excel(file_path)
print("Data loaded successfully. First few rows:")
print(df.head())
except Exception as e:
print(f"Error loading Excel file: {e}")
exit()

# Step 2: Validate required columns

required_columns = ['Year', 'LIT_RATE', 'SEX_RATIO',
'PROP_WORK']
if not all(col in df.columns for col in required_columns):
print(f"Error: Missing required columns. Found columns:{df.columns}")
exit()

# Step 3: Logistic transformation for LIT_RATE
# logit(p) = log(p / (100-p))
df['LOGIT_LIT_RATE'] = np.log(df['LIT_RATE'] / (100 -
df['LIT_RATE']))

# Step 4: Function to perform ARIMA forecasting
def arima_forecast(series, series_name, years,
forecast_steps=19, order=(0, 2, 0), logit_transformed=False):
working_series = series.copy()

# Second differencing
series_diff1 = working_series.diff()
series_diff2 = series_diff1.diff()

# ADF Test on Second Differenced Series
print(f"\nADF Test on Second Differenced {series_name}:")
result = adfuller(series_diff2.dropna())
print('ADF Statistic:', result[0])
print('p-value:', result[1])
print('Critical Values:', result[4])


if result[1] < 0.05:
print(f"Second differenced {series_name} is
stationary")
else:

print(f"Second differenced {series_name} is still non-
stationary")

# ARIMA with d=2
model = ARIMA(working_series, order=order)
fit = model.fit()

# Forecast with 95% and 50% confidence intervals
forecast_obj = fit.get_forecast(steps=forecast_steps)
forecast = forecast_obj.predicted_mean
conf_int_95 = forecast_obj.conf_int(alpha=0.05) # 95%
confidence intervals
conf_int_50 = forecast_obj.conf_int(alpha=0.50) # 50%
confidence intervals
forecast_years = range(2012, 2012 + forecast_steps)

# If logit-transformed, back-transform to original scale:
p = 100 / (1 + exp(-logit))
if logit_transformed:
plot_series = 100 / (1 + np.exp(-series))
plot_forecast = 100 / (1 + np.exp(-forecast))
plot_conf_int_95 = 100 / (1 + np.exp(-conf_int_95))
plot_conf_int_50 = 100 / (1 + np.exp(-conf_int_50))
else:
plot_series = series
plot_forecast = forecast
plot_conf_int_95 = conf_int_95
plot_conf_int_50 = conf_int_50

# Plot with 50% CI for better visualization
plt.figure(figsize=(10, 6))
plt.plot(years, plot_series, marker='o', label='Observed',
color='blue')
plt.plot(forecast_years, plot_forecast, marker='x',
linestyle='--', color='red', label='Forecast')
plt.fill_between(forecast_years, plot_conf_int_50.iloc[:,
0], plot_conf_int_50.iloc[:, 1], color='red', alpha=0.1,
label='50% Confidence Interval')
plt.title(f'ARIMA(0,2,0) Forecast for {series_name} (2012–
2030)')
plt.xlabel('Year')
plt.ylabel(series_name)
plt.legend()
plt.show()

# Print forecast
print(f"\nForecasted {series_name} (2012–2030):")
print(pd.DataFrame({'Year': forecast_years,
f'{series_name}_Forecast': plot_forecast}))
print(f"\nModel Summary for {series_name}:")
print(fit.summary())


return plot_forecast, plot_conf_int_95, plot_conf_int_50

# Step 5: Apply ARIMA to each factor
years = df['Year']
forecast_steps = 19 # Forecast from 2012 to 2030

# Sex Ratio
print("\n=== Forecasting SEX_RATIO ===")
sex_ratio_forecast, sex_ratio_conf_int_95,
sex_ratio_conf_int_50 = arima_forecast(df['SEX_RATIO'],
'SEX_RATIO', years, forecast_steps)

# Literacy Rate (using logit-transformed data)
print("\n=== Forecasting LIT_RATE ===")
lit_rate_forecast, lit_rate_conf_int_95, lit_rate_conf_int_50 = arima_forecast(df['LOGIT_LIT_RATE'], 'LIT_RATE (%)', years,
                                                                               forecast_steps, logit_transformed=True)

# Proportion of Workers
print("\n=== Forecasting PROP_WORK ===")
prop_work_forecast, prop_work_conf_int_95,
prop_work_conf_int_50 = arima_forecast(df['PROP_WORK'],'PROP_WORK (%)', years, forecast_steps)

# Step 6: Print forecasted values for 2030 with confidence
intervals
forecast_years = range(2012, 2031)
forecast_df = pd.DataFrame({
'Year': forecast_years,
'SEX_RATIO_Forecast': sex_ratio_forecast,
'SEX_RATIO_Lower_95': sex_ratio_conf_int_95.iloc[:, 0],
'SEX_RATIO_Upper_95': sex_ratio_conf_int_95.iloc[:, 1],
'SEX_RATIO_Lower_50': sex_ratio_conf_int_50.iloc[:, 0],
'SEX_RATIO_Upper_50': sex_ratio_conf_int_50.iloc[:, 1],
'LIT_RATE_Forecast': lit_rate_forecast,
'LIT_RATE_Lower_95': lit_rate_conf_int_95.iloc[:, 0],
'LIT_RATE_Upper_95': lit_rate_conf_int_95.iloc[:, 1],
'LIT_RATE_Lower_50': lit_rate_conf_int_50.iloc[:, 0],
'LIT_RATE_Upper_50': lit_rate_conf_int_50.iloc[:, 1],
'PROP_WORK_Forecast': prop_work_forecast,
'PROP_WORK_Lower_95': prop_work_conf_int_95.iloc[:, 0],
'PROP_WORK_Upper_95': prop_work_conf_int_95.iloc[:, 1],
'PROP_WORK_Lower_50': prop_work_conf_int_50.iloc[:, 0],
'PROP_WORK_Upper_50': prop_work_conf_int_50.iloc[:, 1]
})

print("\nForecasted Values for 2030 with Confidence
Intervals:")
print(f"Sex Ratio: {forecast_df.loc[forecast_df['Year'] ==
2030, 'SEX_RATIO_Forecast'].values[0]:.2f} "
f"(95% CI: {forecast_df.loc[forecast_df['Year'] == 2030,
'SEX_RATIO_Lower_95'].values[0]:.2f} - "
f"{forecast_df.loc[forecast_df['Year'] == 2030,
'SEX_RATIO_Upper_95'].values[0]:.2f}; "
f"50% CI: {forecast_df.loc[forecast_df['Year'] == 2030,
'SEX_RATIO_Lower_50'].values[0]:.2f} - "
f"{forecast_df.loc[forecast_df['Year'] == 2030,
'SEX_RATIO_Upper_50'].values[0]:.2f})")
print(f"Literacy Rate: {forecast_df.loc[forecast_df['Year'] ==
2030, 'LIT_RATE_Forecast'].values[0]:.2f}% "
f"(95% CI: {forecast_df.loc[forecast_df['Year'] == 2030,
'LIT_RATE_Lower_95'].values[0]:.2f}% - "
f"{forecast_df.loc[forecast_df['Year'] == 2030,
'LIT_RATE_Upper_95'].values[0]:.2f}%; "
f"50% CI: {forecast_df.loc[forecast_df['Year'] == 2030,
'LIT_RATE_Lower_50'].values[0]:.2f}% - "
f"{forecast_df.loc[forecast_df['Year'] == 2030,
'LIT_RATE_Upper_50'].values[0]:.2f}%)")
print(f"Proportion of Workers:
{forecast_df.loc[forecast_df['Year'] == 2030,
'PROP_WORK_Forecast'].values[0]:.2f}% "
f"(95% CI: {forecast_df.loc[forecast_df['Year'] == 2030,
'PROP_WORK_Lower_95'].values[0]:.2f}% - "
f"{forecast_df.loc[forecast_df['Year'] == 2030,
'PROP_WORK_Upper_95'].values[0]:.2f}%; "
f"50% CI: {forecast_df.loc[forecast_df['Year'] == 2030,
'PROP_WORK_Lower_50'].values[0]:.2f}% - "
f"{forecast_df.loc[forecast_df['Year'] == 2030,
'PROP_WORK_Upper_50'].values[0]:.2f}%)")